In [114]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [23]:
vc_invest1 = pd.read_csv("../data/processed/PitchBook_CA_VCInvest=1_NLP-features.csv")

In [24]:
vc_invest0 = pd.read_csv("../data/processed/PitchBook_CA_VCInvest=0_NLP-features.csv")

In [25]:
vc_invest1.head(2)

,Unnamed: 0,Unnamed: 0.1,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Active Investors,...,anger,disgust,fear,joy,sadness,agreeableness_big5,conscientiousness_big5,emotional_range_big5,extraversion_big5,openness_big5
0,0,0,61743-97,Developer of an online advertisement platform ...,ZypMedia,94102,Social/Platform Software,Mark Goldman,2013.0,"Sinclair Broadcast Group, Sinclair Digital Ven...",...,0.037795,0.056217,0.033382,0.331294,0.248749,0.770552,0.554384,0.463544,0.771254,0.440335
1,1,6,62229-34,"Developer of animated, interactive learning ma...",Zyante,95031,Publishing,Smita Bakshi,2012.0,"Bialla Venture Partners, National Science Foun...",...,0.076046,0.045468,0.053601,0.532597,0.353646,0.608447,0.346789,0.447831,0.567102,0.228463


In [26]:
vc_invest1['vc_invest'] = vc_invest1.VC_invested

In [27]:
vc_invest1.drop('VC_invested',axis=1, inplace=True) # make the columns match

In [28]:
vc_invest0 .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 38 columns):
Unnamed: 0                     92 non-null int64
Unnamed: 0.1                   92 non-null int64
Company ID                     92 non-null object
Description                    92 non-null object
Company Name                   92 non-null object
HQ Post Code                   92 non-null int64
Primary Industry Code          92 non-null object
Primary Contact                92 non-null object
Year Founded                   92 non-null float64
Active Investors               56 non-null object
HQ Location                    92 non-null object
Growth Rate                    92 non-null float64
Size Multiple                  92 non-null float64
Last Financing Date            92 non-null object
Last Financing Size            92 non-null float64
Last Financing Valuation       17 non-null float64
Last Financing Deal Type 2     54 non-null object
Majestic Referring Domains     92 non-null fl

In [29]:
vc_invest0['Primary Industry Code'].unique() #  one hot encode this

array(['Network Management Software', 'Electronics (B2C)',
       'Other Business Products and Services', 'Internet Retail',
       'Therapeutic Devices', 'Social/Platform Software',
       'Application Software', 'Social Content', 'Human Capital Services',
       'Automotive', 'Movies, Music and Entertainment',
       'Real Estate Services (B2C)', 'Other Healthcare Services',
       'Other Healthcare Technology Systems', 'Other Commercial Services',
       'Information Services (B2C)', 'Consulting Services (B2B)',
       'Other Software', 'Other Consumer Products and Services',
       'Business/Productivity Software', 'Other Financial Services',
       'Entertainment Software', 'Other Energy Services',
       'Other Consumer Durables', 'Aerospace and Defense',
       'Broadcasting, Radio and Television', 'Leisure Facilities',
       'Wireless Service Providers', 'Legal Services (B2C)',
       'Educational and Training Services (B2C)',
       'Media and Information Services (B2B)',
   

# Combine dataframes

In [30]:
combo_df = pd.concat(objs=[vc_invest0,vc_invest1])

In [31]:
combo_df.tail()

,Active Investors,Company ID,Company Name,Description,Employees,Facebook Likes,Growth Rate,HQ Location,HQ Post Code,Last Financing Date,...,days_since_offer,disgust,emotional_range_big5,extraversion_big5,fear,joy,openness_big5,sadness,tentative,vc_invest
161,"Anjan Mitra, Dorm Room Fund, Emily Mitra, Erik...",106663-87,Traveling Spoon,Developer of an online platform designed to bo...,2.0,19126.0,0.416151,"San Francisco, CA",94110,2015-09-17,...,NaN,0.047489,0.703120,0.890312,0.044991,0.693290,0.630364,0.088702,0.460577,1
162,"Accel, New Enterprise Associates",167364-55,TravelBank,Developer of travel and expense software for s...,11.0,916.0,4.039472,"San Francisco, CA",94107,2016-10-20,...,NaN,0.057233,0.811919,0.889738,0.054336,0.606031,0.136577,0.099967,0.072408,1
163,NaN,58446-37,Translucent Medical,Provider of a navigation system for reducing X...,27.0,1135.0,0.042641,"Santa Cruz, CA",95060,NaN,...,NaN,0.084686,0.765961,0.665861,0.105098,0.581602,0.399683,0.464961,0.380504,1
164,"Amplify.LA, Canyon Creek Capital, HillsVen Cap...",90366-58,Translation Exchange,Provider of a cloud-based platform for continu...,11.0,200.0,3.822225,"San Francisco, CA",94105,2016-03-01,...,NaN,0.248178,0.750920,0.300239,0.127519,0.298173,0.264759,0.369778,0.689846,1
165,"1776, Alex Bresler, BallstonBID LaunchPad, Bru...",95609-35,TransitScreen,Provider of a real-time transportation display...,18.0,774.0,0.135018,"Washington, DC",20005,2017-02-24,...,NaN,0.082909,0.575228,0.623034,0.094825,0.481402,0.468479,0.445690,0.346413,1


In [32]:
combo_df.head()

,Active Investors,Company ID,Company Name,Description,Employees,Facebook Likes,Growth Rate,HQ Location,HQ Post Code,Last Financing Date,...,days_since_offer,disgust,emotional_range_big5,extraversion_big5,fear,joy,openness_big5,sadness,tentative,vc_invest
0,"500 Startups, Start Smart Labs",170123-59,Zyudly Labs,Provider of cyber-security analytics and intel...,7.0,103.0,0.118048,"Palo Alto, CA",94306,2017-02-08,...,121.0,0.100205,0.733407,0.632116,0.377408,0.514982,0.410378,0.283647,0.274317,0
1,eLab Accelerator,123484-96,Zunn Labs,Developer of application controlled LED lights...,7.0,846.0,-0.021589,"Los Angeles, CA",90025,2014-12-20,...,902.0,0.049665,0.779365,0.711512,0.145806,0.339845,0.164799,0.363050,0.319869,0
2,NaN,170046-01,Zuma 4 Productions,The company is currently operating in Stealth ...,7.0,816.0,0.000000,"Malibu, CA",90265,2017-01-26,...,134.0,0.031524,0.464414,0.785069,0.054562,0.769597,0.305475,0.179980,0.364754,0
3,NaN,102923-56,Zulu,Provider of a community-based grocery shopping...,6.0,21.0,0.000000,"Sunnyvale, CA",94085,2014-06-01,...,1104.0,0.059154,0.267228,0.304023,0.097972,0.203800,0.136727,0.127828,0.000000,0
4,California Institute for Quantitative Bioscien...,123222-43,ZoneOne Pharma,Developer of nano-scale therapeutics. The comp...,7.0,816.0,0.000000,"San Francisco, CA",94158,2015-05-01,...,770.0,0.043056,0.532335,0.665466,0.052714,0.502585,0.713422,0.101492,0.448915,0


In [38]:
combo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258 entries, 0 to 165
Data columns (total 38 columns):
Active Investors               219 non-null object
Company ID                     258 non-null object
Company Name                   258 non-null object
Description                    258 non-null object
Employees                      258 non-null float64
Facebook Likes                 258 non-null float64
Growth Rate                    258 non-null float64
HQ Location                    258 non-null object
HQ Post Code                   258 non-null object
Last Financing Date            243 non-null object
Last Financing Deal Type 2     140 non-null object
Last Financing Size            258 non-null float64
Last Financing Valuation       17 non-null float64
Majestic Referring Domains     258 non-null float64
Primary Contact                258 non-null object
Primary Industry Code          258 non-null object
Size Multiple                  258 non-null float64
Total Raised          

# Select columns for the data model
- Company ID  (primary key)
- Employees
- Facebook Likes
- Growth Rate
- HQ Post Code
- Last Financing Size
- Majestic Referring Domains
- Primary Industry Code( use PD get dummies to use in model)
- Size Multiple
- Total Raised
- Twitter Followers
- Year Founded
- agreeableness_big5
- analytical
- anger
- confident
- conscientiousness_big5
- disgust
- emotional_range_big5
- extraversion_big5
- fear
- joy 
- openness_big5
- sadness
- tentative
- vc_invest (columns to predict)

In [68]:
training_df = combo_df[['Company ID', 'Employees','Facebook Likes','Growth Rate','HQ Post Code','Last Financing Size',
         'Majestic Referring Domains','Primary Industry Code','Size Multiple','Total Raised','Twitter Followers',
         'Year Founded','agreeableness_big5','analytical','anger','confident','conscientiousness_big5',
         'disgust','emotional_range_big5','extraversion_big5','fear','joy','openness_big5','sadness','tentative',
         'vc_invest']]

### Turn HQ Post Code into an int

In [69]:
hq_post_code = []
cleaned_final_df = training_df .copy()
for row in training_df['HQ Post Code']:
    try:
        hq_post_code.append(int(row))
    except:
        hq_post_code.append("NaN")

In [70]:
cleaned_final_df ['HQ_post_code'] = hq_post_code

In [71]:
cleaned_final_df .drop(['HQ Post Code'],axis=1,inplace=True)

In [72]:
cleaned_final_df = cleaned_final_df[cleaned_final_df['HQ_post_code']!='NaN']

In [77]:
 cleaned_final_df.HQ_post_code = cleaned_final_df.HQ_post_code.astype('int64')

In [85]:
 cleaned_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 0 to 165
Data columns (total 26 columns):
Company ID                    252 non-null object
Employees                     252 non-null float64
Facebook Likes                252 non-null float64
Growth Rate                   252 non-null float64
Last Financing Size           252 non-null float64
Majestic Referring Domains    252 non-null float64
Primary Industry Code         252 non-null object
Size Multiple                 252 non-null float64
Total Raised                  252 non-null float64
Twitter Followers             252 non-null float64
Year Founded                  252 non-null float64
agreeableness_big5            252 non-null float64
analytical                    252 non-null float64
anger                         252 non-null float64
confident                     252 non-null float64
conscientiousness_big5        252 non-null float64
disgust                       252 non-null float64
emotional_range_big5          

## Add a one hot encoding for the primary industry

In [86]:
final_training_df = pd.get_dummies(cleaned_final_df['Primary Industry Code'])

In [95]:
training_df = pd.concat(objs=[cleaned_final_df,final_training_df],axis=1)

In [98]:
cols = training_df.columns

In [103]:
# move vc_invest to the first columns
training_df = training_df[['vc_invest','Company ID', 'Employees', 'Facebook Likes', 'Growth Rate',
       'Last Financing Size', 'Majestic Referring Domains',
       'Primary Industry Code', 'Size Multiple', 'Total Raised',
       'Twitter Followers', 'Year Founded', 'agreeableness_big5', 'analytical',
       'anger', 'confident', 'conscientiousness_big5', 'disgust',
       'emotional_range_big5', 'extraversion_big5', 'fear', 'joy',
       'openness_big5', 'sadness', 'tentative', 'HQ_post_code',
       'Aerospace and Defense', 'Alternative Energy Equipment',
       'Application Software', 'Application Specific Semiconductors',
       'Automation/Workflow Software', 'Automotive', 'Beverages',
       'Biotechnology', 'Broadcasting, Radio and Television',
       'Business/Productivity Software', 'Communication Software',
       'Consulting Services (B2B)', 'Consumer Finance', 'Database Software',
       'Diagnostic Equipment', 'Drug Discovery',
       'Education and Training Services (B2B)',
       'Educational and Training Services (B2C)', 'Electrical Equipment',
       'Electronic Components', 'Electronics (B2C)', 'Energy Exploration',
       'Energy Production', 'Enterprise Systems (Healthcare)',
       'Entertainment Software', 'Environmental Services (B2B)',
       'Financial Software', 'Food Products', 'General Purpose Semiconductors',
       'Hotels and Resorts', 'Human Capital Services',
       'Information Services (B2C)', 'Internet Retail', 'Legal Services (B2C)',
       'Leisure Facilities', 'Logistics',
       'Media and Information Services (B2B)',
       'Movies, Music and Entertainment', 'Multimedia and Design Software',
       'Network Management Software', 'Other Business Products and Services',
       'Other Commercial Products', 'Other Commercial Services',
       'Other Consumer Durables', 'Other Consumer Products and Services',
       'Other Devices and Supplies', 'Other Energy Services',
       'Other Financial Services', 'Other Healthcare Services',
       'Other Healthcare Technology Systems', 'Other IT Services',
       'Other Information Technology', 'Other Restaurants, Hotels and Leisure',
       'Other Services (B2C Non-Financial)', 'Other Software',
       'Other Transportation', 'Personal Products', 'Private Equity',
       'Publishing', 'Real Estate Services (B2C)', 'Recreational Goods',
       'Restaurants and Bars', 'Security Services (B2B)', 'Social Content',
       'Social/Platform Software', 'Specialty Retail', 'Surgical Devices',
       'Therapeutic Devices', 'Vertical Market Software', 'Water Utilities',
       'Wireless Service Providers']]

# train test split

In [105]:
x_all, y_all = training_df.iloc[:,1:], training_df.iloc[:,0]

In [109]:
X_train, X_test, y_train, y_test  = train_test_split(x_all,y_all,test_size=.2)

In [113]:
X_train.shape # 201 training examples

(201, 96)

In [122]:
X_train.values.shape

(201, 96)

# train random forest

In [115]:
rf_classify = RandomForestClassifier(n_estimators=300)

In [119]:
rf_classify.fit(X_train.values,y_train.values)

ValueError: could not convert string to float: 'Media and Information Services (B2B)'